<a href="https://colab.research.google.com/github/tap-o-chek/Multiple-processing-course/blob/main/MP_Lesson_6_%D0%96%D0%B8%D0%B4%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0_%D0%A1%D0%BE%D1%84%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Упражнение 1.** Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

%time
x = inc(1)
y = inc(2)
z = add(x, y)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [ ]:
import dask
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

x = dask.delayed(inc)(1)
y = dask.delayed(inc)(2)
z = dask.delayed(add)(x, y)

%time z.compute()

CPU times: user 418 ms, sys: 40.7 ms, total: 459 ms
Wall time: 2.55 s


5

Объяснение: распараллеливание позволяет эффективнее использовать ресурсы процессора и сокращает общее время выполнения задач.

# **Упражнение 2.**
Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [ ]:
%time
def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results = []
for x in data:
    if is_even(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)

total = sum(results)
print(total)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.63 µs
90


In [ ]:
import dask
from time import sleep, time

def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

start_time = time()

results = []
for x in data:
    if is_even(x):
        y = dask.delayed(double)(x)  # Задержка вызова функции double
    else:
        y = dask.delayed(inc)(x)     # Задержка вызова функции inc
    results.append(y)

total = dask.delayed(sum)(results)   # Задержка вычисления суммы
print(total.compute())

end_time = time()
wall_time = end_time - start_time
print("Wall time:", wall_time)

90
Wall time: 5.030089855194092


Объяснение: распараллеливание позволяет эффективнее использовать ресурсы процессора и сокращает общее время выполнения задач.

# **Упражнение 3.** Использовать dask.bag.

Мы будем использовать пакет Dask для расчета частот последовательностей по пять символов, а затем сортировать последовательности в порядке убывания частоты встреч.

Реализуйте функцию group_characters(line, n=5) для группировки строки по n символов и возврата итератора. Строка — это текстовая строка в файле genome.txt.  (сгенерировать самостоятельно)

In [ ]:
# >>> line = "abcdefghijklmno"
# >>> for seq in group_character(line, 5):
#         print(seq)

# "abcde"
# "efghi"
# "klmno"

Реализовать group_and_split(line)

In [ ]:
# >>> group_and_split('abcdefghijklmno')
# ['abcde', 'fghij', 'klmno']

Использовать dask.bag для вычисления частот встреч групп символов по 5 и отсортировать по ней.

In [ ]:
from dask import bag as db

def group_characters(line, n=5):
    return (line[i:i+n] for i in range(0, len(line), n))

line = "abcdefghijklmno"
for seq in group_characters(line, 5):
    print(seq)

abcde
fghij
klmno


In [ ]:
# генерация строки в genome.txt
import random
import string

def generate_random_string(length):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

random_string = generate_random_string(100)

file_name = "genome.txt"
with open(file_name, "w") as file:
    file.write(random_string)

In [ ]:
from dask import bag as db

def group_characters(line, n=5):
    return (line[i:i+n] for i in range(0, len(line), n))

file_path = "genome.txt"
lines_bag = db.read_text(file_path)

sequences_bag = lines_bag.map(group_characters)
flattened_bag = sequences_bag.flatten()
frequency_bag = flattened_bag.frequencies()

# Сортировка по убыванию частоты и берем первые 10
sorted_sequences = frequency_bag.topk(10, key=lambda x: x[1]).compute()

print(sorted_sequences)

[('P8aIZ', 1), ('ADb6h', 1), ('EOZZ7', 1), ('tJMRh', 1), ('UV21l', 1), ('P3F9g', 1), ('igTWX', 1), ('wNRE5', 1), ('2ypyc', 1), ('bFIU3', 1)]


# **Упражнение 4.** Использовать dask.array

Вычислите сумму, произведение и среднее значение двумерного квадратного массива (сгенерировать случайно) из 1_000_000 элементов в каждом измерении.

1) с помощью dask (определить самостоятельно разбиение на блоки)

2) без dask

Измерить время

In [ ]:
# Вариант с dask
import dask.array as da
import numpy as np
import time

# Генерируем случайный двумерный массив из 1_000_000 элементов в каждом измерении
np_array = np.random.rand(1000, 1000)

# Используем dask.array для создания dask массива
dask_array = da.from_array(np_array, chunks=(100, 100))

start_time = time.time()

# Вычисляем сумму, произведение и среднее значение
sum_dask = dask_array.sum().compute()
prod_dask = dask_array.prod().compute()
mean_dask = dask_array.mean().compute()

dask_time = time.time() - start_time

print("Dask:")
print("Sum:", sum_dask)
print("Product:", prod_dask)
print("Mean:", mean_dask)
print("Time:", dask_time)

Dask:
Sum: 500415.5843750101
Product: 0.0
Mean: 0.5004155843750101
Time: 0.13397431373596191


In [ ]:
# Без использования dask
start_time = time.time()

# Вычисляем сумму, произведение и среднее значение
sum_np = np_array.sum()
prod_np = np_array.prod()
mean_np = np_array.mean()

np_time = time.time() - start_time

print("\nWithout Dask:")
print("Sum:", sum_np)
print("Product:", prod_np)
print("Mean:", mean_np)
print("Time:", np_time)


Without Dask:
Sum: 500415.58437501005
Product: 0.0
Mean: 0.50041558437501
Time: 0.004935503005981445


# **Упражнение 5.**

Постройте массив 20000x20000 нормально распределенных случайных значений (random), разбитых на фрагменты размером 1000x1000
Возьмите среднее по одной оси
Возьмите каждый 100-й элемент

Сделайте задание в двух вариантах:

1) dask

2) numpy

Измерите время и объясните результат.

In [ ]:
# Вариант с dask
import dask.array as da
import dask

# Генерируем dask массив нормально распределенных случайных значений
dask_array = da.random.normal(size=(20000, 20000),
                              chunks=(1000, 1000))

axis_mean_dask = dask_array.mean(axis=1)

every_100th_element_dask = axis_mean_dask[::100]

start_time = time.time()

print("Dask:")
print("Shape of every_100th_element:", every_100th_element_dask.compute().shape)
print("Time:", time.time() - start_time)

Dask:
Shape of every_100th_element: (200,)
Time: 11.44222378730774


In [ ]:
# Вариант с numpy
import numpy as np
import time

np_array = np.random.randn(20000, 20000)

axis_mean = np_array.mean(axis=1)

every_100th_element = axis_mean[::100]

start_time = time.time()

print("NumPy:")
print("Shape of every_100th_element:", every_100th_element.shape)
print("Time:", time.time() - start_time)

NumPy:
Shape of every_100th_element: (200,)
Time: 0.0017724037170410156


Объяснение: Время выполнения варианта с NumPy значительно меньше по сравнению с вариантом с Dask. Это происходит из-за того, что NumPy оперирует с массивом данных, хранящимся в памяти, и выполняет все операции на одном процессе. то есть, время выполнения ограничено только скоростью доступа к памяти и вычислительной мощности процессора.

Dask же автоматически разбивает массив данных на блоки и распределяет вычисления по нескольким процессорам или даже узлам кластера. В этом примере, хотя и выполняется простая операция - взятие среднего и выбор каждого 100-го элемента, Dask всё равно тратит больше времени на организацию параллельных вычислений, взаимодействие с блоками данных и сбор результатов.